# Testing extracting OSM data using Osmium

In [33]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

In [34]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from GOSTnets.load_traffic2 import *

In [36]:
#read_nodes = pd.read_csv('./vavuniya_unclean_nodes.csv')

In [37]:
#read_nodes[:5]

In [38]:
# read graph
G = nx.read_gpickle('./vavuniya_unclean2.pickle')

In [39]:
len(G.edges)

392096

In [40]:
gn.example_edge(G, 5)

(60796641, 2419600689, {'osm_id': 8111662, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff6bc90>, 'infra_type': 'tertiary', 'min_speed': 6.0, 'max_speed': 30.0, 'mean_speed': 22.518849206349206, 'length': 0.047109297999932145})
(60796641, 2419532369, {'osm_id': 143399474, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff6be50>, 'infra_type': 'tertiary', 'min_speed': 6.0, 'max_speed': 38.0, 'mean_speed': 25.92609126984127, 'length': 0.0941376153234959})
(60796641, 60797501, {'osm_id': 8111676, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff6bf10>, 'infra_type': 'tertiary', 'min_speed': 11.0, 'max_speed': 34.0, 'mean_speed': 25.85218253968254, 'length': 0.06881874072993506})
(2419600689, 3830183348, {'osm_id': 8111662, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff6bf90>, 'infra_type': 'tertiary', 'min_speed': 27.0, 'max_speed': 42.0, 'mean_speed': 31.36904761904762, 'length': 0.03166667608222142})
(241960

In [41]:
G[3825202158][3825202157]

AtlasView({0: {'osm_id': 8111663, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff71250>, 'infra_type': 'trunk', 'min_speed': 9.0, 'max_speed': 40.0, 'mean_speed': 27.865079365079364, 'length': 0.00498046297582135}})

In [42]:
G[3825202157][3825202158]

AtlasView({0: {'osm_id': 8111663, 'geometry': <shapely.geometry.linestring.LineString object at 0x19ff71250>, 'infra_type': 'trunk', 'min_speed': 9.0, 'max_speed': 40.0, 'mean_speed': 27.865079365079364, 'length': 0.00498046297582135}})

In [43]:
len(G.edges)

392096

In [44]:
gn.example_node(G)

(60796641, {'geometry': <shapely.geometry.point.Point object at 0x11fa2a6d0>})


In [48]:
gdfnodes = gn.node_gdf_from_graph(G)

In [ ]:
gdfnodes

In [50]:
non_traffic_gdfnodes = gdfnodes.copy()

In [51]:
non_traffic_gdfnodes

,node_ID,geometry
0,60796641,POINT (79.86617 6.90957)
1,2419600689,POINT (79.86659 6.90955)
2,3830183348,POINT (79.86688 6.90955)
3,91030247,POINT (79.84231 6.93132)
4,3825202158,POINT (79.84231 6.93126)
...,...,...
210114,7369784221,POINT (79.92732 7.02581)
210115,7369784225,POINT (79.92730 7.02577)
210116,7369784284,POINT (79.92297 7.02520)
210117,7369784268,POINT (79.92279 7.02389)


In [83]:
drop_list = []
for u, v, data in G.edges(data = True):
      if data['mean_speed'] > 0:
          drop_list.append(u)
          drop_list.append(v)

drop_list = list(set(drop_list))

In [84]:
non_traffic_gdfnodes = non_traffic_gdfnodes[~non_traffic_gdfnodes['node_ID'].isin(drop_list)]

In [85]:
non_traffic_gdfnodes

,node_ID,geometry
21,4047124003,POINT (79.87387 6.90471)
73,50422994,POINT (79.86590 6.85131)
211,1863456369,POINT (79.85125 6.93425)
215,91031378,POINT (79.85225 6.93613)
220,91170617,POINT (79.84987 6.93454)
...,...,...
210114,7369784221,POINT (79.92732 7.02581)
210115,7369784225,POINT (79.92730 7.02577)
210116,7369784284,POINT (79.92297 7.02520)
210117,7369784268,POINT (79.92279 7.02389)


In [53]:
node_list = []
for u, v, data in G.edges(data = True):
      if not data['mean_speed'] > 0:
        node_list.append(u)
        node_list.append(v)

In [54]:
len(list(set(node_list)))

162202

In [56]:
#G.nodes[60796641]['geometry']

In [57]:
G.nodes[3727426632]

In [75]:
for successor in G.successors(3345099186):
    print(successor)

5559025764
3345099190
1645495725


In [77]:
for successor in G.successors(3477910642):
    print(successor)

7037574231
5578292935
2028077011
2966142449


In [101]:
G_clean = gn.clean_network(G, UTM = 'epsg:32644', WGS = 'epsg:4326', junctdist = 10, verbose = False)

done generating node list of non-mapbox traffic nodes
print gdfnodes_proj_buffer
           node_ID                       geometry
21      4047124003  POINT (375585.124 763361.787)
73        50422994  POINT (374690.576 757460.121)
211     1863456369  POINT (373093.228 766633.620)
215       91031378  POINT (373204.807 766841.871)
220       91170617  POINT (372940.976 766666.076)
...            ...                            ...
210114  7369784221  POINT (381521.640 776737.741)
210115  7369784225  POINT (381519.300 776733.601)
210116  7369784284  POINT (381040.061 776671.125)
210117  7369784268  POINT (381020.021 776526.583)
210118  7369784287  POINT (380949.728 776913.583)

[147845 rows x 2 columns]
print juncs_gdf
                                                geometry
0      POLYGON ((389622.777 742802.353, 389622.729 74...
1      POLYGON ((389621.976 742843.263, 389621.928 74...
2      POLYGON ((389619.520 742885.336, 389619.471 74...
3      POLYGON ((389601.001 742937.537, 389600.9

NameError: name 'c' is not defined

In [63]:
list_1 = [1,2,3,4,5]

In [64]:
list_1[:-1]

[1, 2, 3, 4]

In [65]:
list_1[1:]

[2, 3, 4, 5]

In [66]:
G[2423783429][2423783444]

AtlasView({0: {'osm_id': 161632760, 'geometry': <shapely.geometry.linestring.LineString object at 0x170dc9210>, 'infra_type': 'residential', 'min_speed': nan, 'max_speed': nan, 'mean_speed': nan, 'length': 0.0073407062793855804}})

In [67]:
G[2423783444][2423783429]

AtlasView({0: {'osm_id': 161632760, 'geometry': <shapely.geometry.linestring.LineString object at 0x170dc9210>, 'infra_type': 'residential', 'min_speed': nan, 'max_speed': nan, 'mean_speed': nan, 'length': 0.0073407062793855804}})